This script will take in a list of patient IDs and grab the patient image data and labels (ground truth) for live model evaluation.

In [46]:
import os
import pandas as pd
import random
import pydicom

In [7]:
df = pd.read_csv("../data/meta_data_B.csv")

In [68]:
df

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,lable
0,2,42528,379528137,L,MLO,68.0,0,0,0,NaN,0,NaN,21,False,Benign
1,2,42528,1912461073,L,MLO,68.0,0,0,0,NaN,0,NaN,21,False,Benign
2,2,42528,2131243012,L,CC,68.0,0,0,0,NaN,0,NaN,21,False,Benign
3,2,42528,564857574,R,MLO,68.0,0,0,0,NaN,0,NaN,21,False,Benign
4,2,42528,884250295,R,CC,68.0,0,0,0,NaN,0,NaN,21,False,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19013,1,65530,1401303823,R,MLO,48.0,0,1,0,0.0,0,C,49,True,Benign
19014,1,65534,1436834199,L,CC,46.0,0,0,0,1.0,0,C,49,False,Benign
19015,1,65534,1888933323,L,MLO,46.0,0,0,0,1.0,0,C,49,False,Benign
19016,1,65534,44480502,R,MLO,46.0,0,0,0,1.0,0,C,49,False,Benign


In [69]:
df["label"] = df["cancer"].map({0:"Benign", 1:"Malignant"})

In [73]:
#The data for each image is stored on "../data/rsna-breast-cancer-detection/train_images/PATIENTID/IMAGE_ID
#Where the all caps words represent the patientID and Image values respectively.
benign_cases = []


benign_count = 0
benign_desired = 7

malignant_count = 0 
malignant_desired = 7


while benign_count < benign_desired:
    #Find a random patient in the row
    random_row = random.randint(0, len(df) - 1)
    data_sample = df.iloc[random_row]
    #Catch the patient ID for this randomly found row and make a sub dataframe only containing this patient ID
    
    patient = df[df["patient_id"] == data_sample["patient_id"]]
    
    print(patient)
    if data_sample["cancer"] == 0 and benign_count < 7:
        for i in range(len(patient)):
            case = {
                "patient_id" : f"{patient.iloc[i]['patient_id']}",
                "image_id" : f"{patient.iloc[i]['image_id']}",
                "laterality" : f"{patient.iloc[i]['laterality']}",
                "view" : f"{patient.iloc[i]['view']}",
                "cancer" : f"{patient.iloc[i]['label']}"
            }    
            source_path = (f"../data/rsna-breast-cancer-detection/train_images/{case['patient_id']}/{case['image_id']}.dcm")
            #benign_cases.append(case)
            #Load dicom and save DICOM to file structure.
            dicom = pydicom.dcmread(source_path)
            
            save_path = (f"../evaluation_patients/{case['patient_id']}/")
            os.makedirs(save_path, exist_ok=True)
            save_path += f"{case['laterality']}_{case['view']}_{case['cancer']}.dcm"
            print(save_path)
            dicom.save_as(save_path)
        benign_count += 1
        
while malignant_count < malignant_desired:
    #Find a random patient in the row
    random_row = random.randint(0, len(df) - 1)
    data_sample = df.iloc[random_row]
    #Catch the patient ID for this randomly found row and make a sub dataframe only containing this patient ID
    
    patient = df[df["patient_id"] == data_sample["patient_id"]]
    
    print(patient)
    if data_sample["cancer"] == 1 and malignant_count < 7:
        for i in range(len(patient)):
            case = {
                "patient_id" : f"{patient.iloc[i]['patient_id']}",
                "image_id" : f"{patient.iloc[i]['image_id']}",
                "laterality" : f"{patient.iloc[i]['laterality']}",
                "view" : f"{patient.iloc[i]['view']}",
                "cancer" : f"{patient.iloc[i]['label']}"
            }    
            source_path = (f"../data/rsna-breast-cancer-detection/train_images/{case['patient_id']}/{case['image_id']}.dcm")
            #benign_cases.append(case)
            #Load dicom and save DICOM to file structure.
            dicom = pydicom.dcmread(source_path)
            #If Monochrome1, then skip.
            if dicom.PhotometricInterpretation == "MONOCHROME1":
                continue
            
            save_path = (f"../evaluation_patients/{case['patient_id']}/")
            os.makedirs(save_path, exist_ok=True)
            save_path += f"{case['laterality']}_{case['view']}_{case['cancer']}.dcm"
            print(save_path)
            dicom.save_as(save_path)
        malignant_count += 1


       site_id  patient_id    image_id laterality view   age  cancer  biopsy  \
10560        1       55509   296303892          L  MLO  80.0       0       0   
10561        1       55509  1202394098          L   CC  80.0       0       0   
10562        1       55509  1808416106          R  MLO  80.0       0       0   
10563        1       55509  2093586818          R   CC  80.0       0       0   

       invasive  BIRADS  implant density  machine_id  difficult_negative_case  \
10560         0     1.0        0       B          49                    False   
10561         0     1.0        0       B          49                    False   
10562         0     1.0        0       B          49                    False   
10563         0     1.0        0       B          49                    False   

        lable   label  
10560  Benign  Benign  
10561  Benign  Benign  
10562  Benign  Benign  
10563  Benign  Benign  
../evaluation_patients/55509/L_MLO_Benign.dcm
../evaluation_patients/5550